In [1]:
import copy
import itertools as it
import numpy as np
import yaml
import os

def expand_config(dict_config):
    keys, values = zip(*dict_config.items())
    permutations_dicts = [dict(zip(keys, v)) for v in it.product(*values)]
    return permutations_dicts

# CLINC, ROSTD training HP search

In [2]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\={} do_ue_estimate\=False'.format(config['ue']))
    
    if config['ue'] != 'sngp':
        config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))

    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    
    config_list.append('training\=electra_base')
    return config_list

In [3]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    j = 0
    n_devices = len(cuda_devices)
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                base_arg = f'CUDA_VISIBLE_DEVICES={cuda_devices[j%n_devices]} HYDRA_CONFIG_PATH=../configs/{task_name}.yaml python ./run_ood_with_hyp_search.py'

                new_task = copy.deepcopy(base_arg)
                args = ' '.join(generate_func[i](conf))
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_sn = 'sn' if conf['use_spectralnorm']=='True' else 'no_sn'
                sn = 'sngp' if conf['ue'] == 'sngp' else use_sn

                new_task += f' {args}'
                new_task += f' hydra.run.dir=../workdir/hp_search_100/electra_raw_{sn}/{task_name}'
                new_task += " model.model_name_or_path='google/electra-base-discriminator'"
                if (j+1)%n_devices!=0: 
                    new_task += ' &'
                else:
                    new_task += '\nwait'
                full_config += '\n' + new_task if len(full_config) else new_task
                j+=1
                
    with open (f'../../scripts/ood_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [4]:
tasks = ['clinc', 'rostd']

train_configs = {
    'ue': ['sngp'],
    'use_spectralnorm': ['True'],
    'reg_type': ['raw'],
}
    
cuda_devices = [0, 1]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_hp_search_sngp_ood.sh')

In [5]:
tasks = ['clinc', 'rostd']

train_configs = {
    'ue': ['mc'],
    'use_spectralnorm': ['True', 'False'],
    'reg_type': ['raw'],
    'n_power_iterations': [100],
}
    
cuda_devices = [0, 1]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_hp_search_ood.sh')

# HP Search 20newsgroups

In [6]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\={} do_ue_estimate\=False'.format(config['ue']))
    
    if config['ue'] != 'sngp':
        config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    
    config_list.append('training\=electra_base')
    return config_list

In [7]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    j = 0
    n_devices = len(cuda_devices)
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                base_arg = f'CUDA_VISIBLE_DEVICES={cuda_devices[j%n_devices]} HYDRA_CONFIG_PATH=../configs/{task_name}.yaml python ./run_ood_with_hyp_search.py'

                new_task = copy.deepcopy(base_arg)
                args = ' '.join(generate_func[i](conf))
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_sn = 'sn' if conf['use_spectralnorm']=='True' else 'no_sn'
                sn = 'sngp' if conf['ue'] == 'sngp' else use_sn

                new_task += f' {args}'
                new_task += f' hydra.run.dir=../workdir/hp_search/electra_raw_{sn}/{task_name}'
                new_task += " model.model_name_or_path='google/electra-base-discriminator'"
                if (j+1)%n_devices!=0: 
                    new_task += ' &'
                else:
                    new_task += '\nwait'
                full_config += '\n' + new_task if len(full_config) else new_task
                j+=1
                
    with open (f'../../scripts/ood_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [8]:
tasks = ['20newsgroups_ood']

train_configs = {
    'ue': ['sngp'],
    'use_spectralnorm': ['True'],
    'reg_type': ['raw'],
}
    
cuda_devices = [0, 1]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_hp_search_sngp_20ng_ood.sh')

In [9]:
tasks = ['20newsgroups_ood']

train_configs = {
    'ue': ['mc'],
    'use_spectralnorm': ['True', 'False'],
    'reg_type': ['raw'],
}
    
cuda_devices = [0, 1]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_hp_search_20ng_ood.sh')

# OOD exps with new benchmark

In [24]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc')
    config_list.append('do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.calibrate\={}'.format(config['calibrate']))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    if config['use_spectralnorm'] == 'True':
        config_list.append('+ue.n_power_iterations\={}'.format(config['n_power_iterations']))
        
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    
    config_list.append('data.validation_subsample\=0.0')
    config_list.append('training\=electra_base')
    return config_list

In [25]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                try:
                    task_configs = f'{task_name}_ood.yaml'

                    new_task = copy.deepcopy(base_arg)
                    new_task += f' cuda_devices={cuda_devices}'
                    script = conf['script']
                    new_task += f' script={script} seeds=[23419,705525,4837,10671619,1084218,43]'
                    
                    if conf['subsample_perc'] == 0.0:
                        if task_name=='sst2':
                            conf['subsample_perc'] = 0.1
                        elif task_name=='20newsgroups':
                            conf['subsample_perc'] = 0.0
                            
                    subsample_perc = conf['subsample_perc']
                    
                    args = ' '.join(generate_func[i](conf))

                    reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                    use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                    use_spectralnorm_params = '_sn' if conf['use_spectralnorm'] == 'True' else ''
                    
                    if task_name == '20newsgroups':
                        path = f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/20newsgroups_ood/optimal_hyperparameters.yaml'
                    else:
                        path = f'../../optimal_hyperparameters/electra/raw{use_spectralnorm_params}_{task_name}_optimal_hyperparameters.yaml'
                    with open(path, 'r') as f:
                        params = yaml.load(f, Loader=yaml.FullLoader)

                    num_train_epochs = params['num_train_epochs']
                    args += f' training.num_train_epochs\={num_train_epochs}'
                    learning_rate = params['learning_rate']
                    args += f' training.learning_rate\={learning_rate}'
                    per_device_train_batch_size = params['per_device_train_batch_size']
                    args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                    weight_decay = params['weight_decay']
                    args += f' +training.weight_decay\={weight_decay}'

                    margin = params['margin']
                    lamb_intra = params['lamb_intra']
                    lamb = params['lamb']
                    args += f' +ue.margin\={margin}'
                    args += f' +ue.lamb_intra\={lamb_intra}'
                    args += f' ue.lamb\={lamb}'

                    new_task += f" args='{args}'"
                    new_task += f' task_configs={task_configs}'

                    new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/subsample_perc_{subsample_perc}'

                    full_config += '\n' + new_task if len(full_config) else new_task
                except:
                    pass
    with open (f'../../scripts/ood_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [26]:
tasks = ['sst2', '20newsgroups']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['raw'], 
    'use_spectralnorm': ['False', 'True'],
    'script': ['run_ood.py'],
    'subsample_perc': [0.001, 0.005, 0.01, 0.015, 0.02, 0.03, 0.0],
    'n_power_iterations': [10],
     #0.0 - means 0.1 for sst2 and 1.0 for 20ng
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_new_benchmark_subsamples.sh')

In [27]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))
    config_list.append('++ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))

    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))

    config_list.append('do_ue_estimate\=True')
    
    if config['ue']=='mc':
        config_list.append('ue.dropout_subs\=all')
    
    if config['ue'] == 'mc_mahalanobis':
        config_list.append('ue.use_cache\=False')
    else:
        config_list.append('ue.use_cache\=True')
    
    config_list.append('ue.calibrate\=False')
    config_list.append('data.ood_data\={}'.format(config['ood_data']))
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    
    if config['ue'] == 'nuq' or config['ue'] == 'l_nuq':
        config_list.append('ue.nuq.n_neighbors\=60')
        config_list.append('ue.nuq.log_pN\=0')
        config_list.append('+ue.nuq.n_points\=50')
        config_list.append('+ue.nuq.n_folds\=20')
        config_list.append('+ue.nuq.n_samples\=5')
    
    config_list.append('data.validation_subsample\=0.0')
    config_list.append('training\=electra_base')
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                if conf['ue'] == 'mc' and conf['use_spectralnorm'] == 'True':
                    continue
                if conf['ue'] == 'l_nuq' and conf['use_spectralnorm'] == 'False':
                    continue
                if conf['ue'] == 'l_mahalanobis' and conf['use_spectralnorm'] == 'False':
                    continue
                try:
                    task_configs = f'{task_name}_ood.yaml'

                    new_task = copy.deepcopy(base_arg)
                    new_task += f' cuda_devices={cuda_devices}'
                    script = conf['script']
                    new_task += f' script={script}'
                    
                    if conf['subsample_perc'] == 0.0:
                        if task_name=='sst2':
                            conf['subsample_perc'] = 0.1
                        elif task_name=='20newsgroups':
                            conf['subsample_perc'] = 0.0
                            
                    subsample_perc = conf['subsample_perc']

                    args = ' '.join(generate_func[i](conf))

                    reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                    use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                    use_spectralnorm_params = '_sn' if conf['use_spectralnorm'] == 'True' else ''
                    subsample_perc = conf['subsample_perc']

                    if task_name == '20newsgroups':
                        path = f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/20newsgroups_ood/optimal_hyperparameters.yaml'
                    else:
                        path = f'../../optimal_hyperparameters/electra/raw{use_spectralnorm_params}_{task_name}_optimal_hyperparameters.yaml'
                    with open(path, 'r') as f:
                        params = yaml.load(f, Loader=yaml.FullLoader)

                    num_train_epochs = params['num_train_epochs']
                    args += f' training.num_train_epochs\={num_train_epochs}'
                    learning_rate = params['learning_rate']
                    args += f' training.learning_rate\={learning_rate}'
                    per_device_train_batch_size = params['per_device_train_batch_size']
                    args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                    weight_decay = params['weight_decay']
                    args += f' +training.weight_decay\={weight_decay}'

                    margin = params['margin']
                    lamb_intra = params['lamb_intra']
                    lamb = params['lamb']
                    args += f' ++ue.margin\={margin}'
                    args += f' ++ue.lamb_intra\={lamb_intra}'
                    args += f' ue.lamb\={lamb}'

                    args += f' training.per_device_eval_batch_size\=256'

                    new_task += f" args='{args}'"
                    new_task += f' config_path=../configs/{task_configs}'

                    reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                    use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                    ood_data = conf['ood_data']
                    if task_name == ood_data:
                        continue

                    base_path=f' output_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/subsample_perc_{subsample_perc}/{ood_data}'

                    ue_type = conf['ue']
                    base_path+= f'/{ue_type}'
                    new_task+=base_path

                    if task_name == '20newsgroups':
                        model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/subsample_perc_{subsample_perc}/models/{task_name}_ood'
                    else:
                        model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/subsample_perc_{subsample_perc}/models/{task_name}'
                    new_task+=f' model_series_dir={model_series_dir}'

                    calc_output_dir = f'../workdir/run_calc_ues_metrics/electra_{reg}_{use_spectralnorm}/{task_name}/subsample_perc_{subsample_perc}/{ood_data}/{ue_type}'
                    calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics.py runs_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/subsample_perc_{subsample_perc}/{ood_data}/{ue_type} extract_config=False output_dir={calc_output_dir};'

                    full_config += '\n' + new_task if len(full_config) else new_task
                    full_config += '\n' + calc_command
                except:
                    pass
    with open (f'../../scripts/ood_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [28]:
tasks = ['sst2', '20newsgroups']

ue_configs = {
    'ue': ['nuq', 'mahalanobis', 'mc', 'l_nuq', 'l_mahalanobis'],
    'use_spectralnorm': ['False', 'True'],
    'reg_type': ['raw'],
    'script': ['run_ood.py'],
    'ood_data': ['imdb', 'trec', 'wmt16', 'amazon', 'mnli', 'rte', '20newsgroups', 'sst2'],
    'subsample_perc': [0.001, 0.005, 0.01, 0.015, 0.02, 0.03, 0.0],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_new_benchmark_subsamples.sh')

In [29]:
full_test = 'bash run_train_electra_models_new_benchmark_subsamples.sh\n'
full_test += 'bash run_eval_electra_new_benchmark_subsamples.sh'

with open (f'../../scripts/ood_scripts/run_exps_new_benchmark_subsamples.sh', 'w') as rsh:
    rsh.write(full_test)

### sngp

In [183]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=sngp')
    config_list.append('do_ue_estimate\=False')

    config_list.append('ue.calibrate\={}'.format(config['calibrate']))
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    
    config_list.append('data.validation_subsample\=0.0')
    config_list.append('training\=electra_base')
    return config_list

In [184]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                try:
                    task_configs = f'{task_name}_ood.yaml'

                    new_task = copy.deepcopy(base_arg)
                    new_task += f' cuda_devices={cuda_devices}'
                    script = conf['script']
                    new_task += f' script={script} seeds=[23419,705525,4837,10671619,1084218,43]'
                    
                    if conf['subsample_perc'] == 0.0:
                        if task_name=='sst2':
                            conf['subsample_perc'] = 0.1
                        elif task_name=='20newsgroups':
                            conf['subsample_perc'] = 0.0
                            
                    subsample_perc = conf['subsample_perc']
                    
                    args = ' '.join(generate_func[i](conf))

                    reg = 'raw' 
                    use_spectralnorm = 'sngp'
                    
                    if task_name == '20newsgroups':
                        path = f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/20newsgroups_ood/optimal_hyperparameters.yaml'
                    else:
                        path = f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/sst2/optimal_hyperparameters.yaml'
                    with open(path, 'r') as f:
                        params = yaml.load(f, Loader=yaml.FullLoader)

                    num_train_epochs = params['num_train_epochs']
                    args += f' training.num_train_epochs\={num_train_epochs}'
                    learning_rate = params['learning_rate']
                    args += f' training.learning_rate\={learning_rate}'
                    per_device_train_batch_size = params['per_device_train_batch_size']
                    args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                    weight_decay = params['weight_decay']
                    args += f' +training.weight_decay\={weight_decay}'
                    
                    new_task += f" args='{args}'"
                    new_task += f' task_configs={task_configs}'

                    new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/subsample_perc_{subsample_perc}'

                    full_config += '\n' + new_task if len(full_config) else new_task
                except:
                    pass
    with open (f'../../scripts/ood_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [185]:
tasks = ['20newsgroups', 'sst2']

train_configs = {
    'calibrate': ['False'],
    'script': ['run_ood.py'],
    'subsample_perc': [0.001, 0.005, 0.01, 0.015, 0.02, 0.03, 0.0],
    #0.0 - means 0.1 for sst2 and 1.0 for 20ng
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_new_benchmark_subsamples_sngp.sh')

In [21]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))

    config_list.append('do_ue_estimate\=True')
    
    config_list.append('ue.calibrate\=False')
    config_list.append('data.ood_data\={}'.format(config['ood_data']))
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    
    config_list.append('data.validation_subsample\=0.0')
    config_list.append('training\=electra_base')
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                try:
                    task_configs = f'{task_name}_ood.yaml'

                    new_task = copy.deepcopy(base_arg)
                    new_task += f' cuda_devices={cuda_devices}'
                    script = conf['script']
                    new_task += f' script={script}'
                    
                    if conf['subsample_perc'] == 0.0:
                        if task_name=='sst2':
                            conf['subsample_perc'] = 0.1
                        elif task_name=='20newsgroups':
                            conf['subsample_perc'] = 0.0
                            
                    subsample_perc = conf['subsample_perc']

                    args = ' '.join(generate_func[i](conf))

                    reg = 'raw'
                    use_spectralnorm = 'sngp'
                    subsample_perc = conf['subsample_perc']

                    if task_name == '20newsgroups':
                        path = f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/20newsgroups_ood/optimal_hyperparameters.yaml'
                    else:
                        path = f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/sst2/optimal_hyperparameters.yaml'
                    with open(path, 'r') as f:
                        params = yaml.load(f, Loader=yaml.FullLoader)

                    num_train_epochs = params['num_train_epochs']
                    args += f' training.num_train_epochs\={num_train_epochs}'
                    learning_rate = params['learning_rate']
                    args += f' training.learning_rate\={learning_rate}'
                    per_device_train_batch_size = params['per_device_train_batch_size']
                    args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                    weight_decay = params['weight_decay']
                    args += f' +training.weight_decay\={weight_decay}'
                    
                    args += f' training.per_device_eval_batch_size\=256'

                    new_task += f" args='{args}'"
                    new_task += f' config_path=../configs/{task_configs}'

                    ood_data = conf['ood_data']
                    if task_name == ood_data:
                        continue

                    base_path=f' output_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/subsample_perc_{subsample_perc}/{ood_data}'

                    ue_type = conf['ue']
                    base_path+= f'/{ue_type}'
                    new_task+=base_path

                    if task_name == '20newsgroups':
                        model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/subsample_perc_{subsample_perc}/models/{task_name}_ood'
                    else:
                        model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/subsample_perc_{subsample_perc}/models/{task_name}'
                    new_task+=f' model_series_dir={model_series_dir}'

                    calc_output_dir = f'../workdir/run_calc_ues_metrics/electra_{reg}_{use_spectralnorm}/{task_name}/subsample_perc_{subsample_perc}/{ood_data}/{ue_type}'
                    calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics.py runs_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/subsample_perc_{subsample_perc}/{ood_data}/{ue_type} extract_config=False output_dir={calc_output_dir};'

                    full_config += '\n' + new_task if len(full_config) else new_task
                    full_config += '\n' + calc_command
                except:
                    pass
    with open (f'../../scripts/ood_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [22]:
tasks = ['sst2', '20newsgroups']

ue_configs = {
    'ue': ['sngp'],
    'script': ['run_ood.py'],
    'ood_data': ['imdb', 'trec', 'wmt16', 'amazon', 'mnli', 'rte', '20newsgroups', 'sst2'],
    'subsample_perc': [0.001, 0.005, 0.01, 0.015, 0.02, 0.03, 0.0],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_new_benchmark_subsamples_sngp.sh')

In [23]:
full_test = 'bash run_train_electra_models_new_benchmark_subsamples_sngp.sh\n'
full_test += 'bash run_eval_electra_new_benchmark_subsamples_sngp.sh'

with open (f'../../scripts/ood_scripts/run_exps_new_benchmark_subsamples_sngp.sh', 'w') as rsh:
    rsh.write(full_test)

In [189]:
full_test = 'bash run_exps_new_benchmark_subsamples.sh\n'
full_test += 'bash run_exps_new_benchmark_subsamples_sngp.sh'

with open (f'../../scripts/ood_scripts/run_all_exps_new_benchmark_subsamples.sh', 'w') as rsh:
    rsh.write(full_test)

### Noise

In [30]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc')
    config_list.append('do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.calibrate\={}'.format(config['calibrate']))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    
    if config['use_spectralnorm'] == 'True':
        config_list.append('+ue.n_power_iterations\={}'.format(config['n_power_iterations']))
        
    config_list.append('+data.noise_perc\={}'.format(config['noise_perc']))    
    config_list.append('data.validation_subsample\=0.0')
    config_list.append('training\=electra_base')
    return config_list

In [31]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                try:
                    task_configs = f'{task_name}_ood.yaml'

                    new_task = copy.deepcopy(base_arg)
                    new_task += f' cuda_devices={cuda_devices}'
                    script = conf['script']
                    new_task += f' script={script} seeds=[23419,705525,4837,10671619,1084218,43]'
                            
                    noise_perc = conf['noise_perc']
                    
                    args = ' '.join(generate_func[i](conf))

                    reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                    use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                    use_spectralnorm_params = '_sn' if conf['use_spectralnorm'] == 'True' else ''
                    
                    if task_name == '20newsgroups':
                        path = f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/20newsgroups_ood/optimal_hyperparameters.yaml'
                    else:
                        path = f'../../optimal_hyperparameters/electra/raw{use_spectralnorm_params}_{task_name}_optimal_hyperparameters.yaml'
                    with open(path, 'r') as f:
                        params = yaml.load(f, Loader=yaml.FullLoader)

                    num_train_epochs = params['num_train_epochs']
                    args += f' training.num_train_epochs\={num_train_epochs}'
                    learning_rate = params['learning_rate']
                    args += f' training.learning_rate\={learning_rate}'
                    per_device_train_batch_size = params['per_device_train_batch_size']
                    args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                    weight_decay = params['weight_decay']
                    args += f' +training.weight_decay\={weight_decay}'

                    margin = params['margin']
                    lamb_intra = params['lamb_intra']
                    lamb = params['lamb']
                    args += f' +ue.margin\={margin}'
                    args += f' +ue.lamb_intra\={lamb_intra}'
                    args += f' ue.lamb\={lamb}'

                    new_task += f" args='{args}'"
                    new_task += f' task_configs={task_configs}'

                    new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}'

                    full_config += '\n' + new_task if len(full_config) else new_task
                except:
                    pass
    with open (f'../../scripts/ood_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [32]:
tasks = ['sst2', '20newsgroups']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['raw'], 
    'use_spectralnorm': ['False', 'True'],
    'script': ['run_ood.py'],
    'noise_perc': [0.01, 0.05, 0.1, 0.15, 0.2, 0.3],
    'n_power_iterations': [10],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_new_benchmark_noise.sh')

In [34]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))
    config_list.append('++ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))

    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))

    config_list.append('do_ue_estimate\=True')
    
    if config['ue']=='mc':
        config_list.append('ue.dropout_subs\=all')
    
    if config['ue'] == 'mc_mahalanobis':
        config_list.append('ue.use_cache\=False')
    else:
        config_list.append('ue.use_cache\=True')
    
    config_list.append('ue.calibrate\=False')
    config_list.append('data.ood_data\={}'.format(config['ood_data']))
    config_list.append('+data.noise_perc\={}'.format(config['noise_perc']))
    
    if config['ue'] == 'nuq' or config['ue'] == 'l_nuq':
        config_list.append('ue.nuq.n_neighbors\=60')
        config_list.append('ue.nuq.log_pN\=0')
        config_list.append('+ue.nuq.n_points\=50')
        config_list.append('+ue.nuq.n_folds\=20')
        config_list.append('+ue.nuq.n_samples\=5')
        
    config_list.append('data.validation_subsample\=0.0')
    config_list.append('training\=electra_base')
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                if conf['ue'] != 'mc' and conf['use_spectralnorm'] == 'False':
                    continue
                if conf['ue'] == 'mc' and conf['use_spectralnorm'] != 'False':
                    continue
                try:
                    task_configs = f'{task_name}_ood.yaml'

                    new_task = copy.deepcopy(base_arg)
                    new_task += f' cuda_devices={cuda_devices}'
                    script = conf['script']
                    new_task += f' script={script}'

                    noise_perc = conf['noise_perc']

                    args = ' '.join(generate_func[i](conf))

                    reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                    use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                    use_spectralnorm_params = '_sn' if conf['use_spectralnorm'] == 'True' else ''

                    if task_name == '20newsgroups':
                        path = f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/20newsgroups_ood/optimal_hyperparameters.yaml'
                    else:
                        path = f'../../optimal_hyperparameters/electra/raw{use_spectralnorm_params}_{task_name}_optimal_hyperparameters.yaml'
                    with open(path, 'r') as f:
                        params = yaml.load(f, Loader=yaml.FullLoader)

                    num_train_epochs = params['num_train_epochs']
                    args += f' training.num_train_epochs\={num_train_epochs}'
                    learning_rate = params['learning_rate']
                    args += f' training.learning_rate\={learning_rate}'
                    per_device_train_batch_size = params['per_device_train_batch_size']
                    args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                    weight_decay = params['weight_decay']
                    args += f' +training.weight_decay\={weight_decay}'

                    margin = params['margin']
                    lamb_intra = params['lamb_intra']
                    lamb = params['lamb']
                    args += f' ++ue.margin\={margin}'
                    args += f' ++ue.lamb_intra\={lamb_intra}'
                    args += f' ue.lamb\={lamb}'

                    args += f' training.per_device_eval_batch_size\=256'

                    new_task += f" args='{args}'"
                    new_task += f' config_path=../configs/{task_configs}'

                    reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                    use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                    ood_data = conf['ood_data']
                    if task_name == ood_data:
                        continue

                    base_path=f' output_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}/{ood_data}'

                    ue_type = conf['ue']
                    base_path+= f'/{ue_type}'
                    new_task+=base_path

                    if task_name == '20newsgroups':
                        model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}/models/{task_name}_ood'
                    else:
                        model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}/models/{task_name}'
                    new_task+=f' model_series_dir={model_series_dir}'

                    calc_output_dir = f'../workdir/run_calc_ues_metrics/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}/{ood_data}/{ue_type}'
                    calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics.py runs_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}/{ood_data}/{ue_type} extract_config=False output_dir={calc_output_dir};'

                    full_config += '\n' + new_task if len(full_config) else new_task
                    full_config += '\n' + calc_command
                except:
                    pass
    with open (f'../../scripts/ood_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [35]:
tasks = ['sst2', '20newsgroups']

ue_configs = {
    'ue': ['nuq', 'mahalanobis', 'mc', 'l_nuq', 'l_mahalanobis'],
    'use_spectralnorm': ['False', 'True'],
    'reg_type': ['raw'],
    'script': ['run_ood.py'],
    'ood_data': ['imdb', 'trec', 'wmt16', 'amazon', 'mnli', 'rte', '20newsgroups', 'sst2'],
    'noise_perc': [0.01, 0.05, 0.1, 0.15, 0.2, 0.3],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_new_benchmark_noise.sh')

In [36]:
full_test = 'bash run_train_electra_models_new_benchmark_noise.sh\n'
full_test += 'bash run_eval_electra_new_benchmark_noise.sh'

with open (f'../../scripts/ood_scripts/run_exps_new_benchmark_noise.sh', 'w') as rsh:
    rsh.write(full_test)

### sngp

In [37]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=sngp')
    config_list.append('do_ue_estimate\=False')
        
    config_list.append('+data.noise_perc\={}'.format(config['noise_perc']))    
    config_list.append('training\=electra_base')
    config_list.append('data.validation_subsample\=0.0')
    return config_list

In [38]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                try:
                    task_configs = f'{task_name}_ood.yaml'

                    new_task = copy.deepcopy(base_arg)
                    new_task += f' cuda_devices={cuda_devices}'
                    script = conf['script']
                    new_task += f' script={script} seeds=[23419,705525,4837,10671619,1084218,43]'
                            
                    noise_perc = conf['noise_perc']
                    
                    args = ' '.join(generate_func[i](conf))

                    reg = 'raw' 
                    use_spectralnorm = 'sngp'
                    
                    if task_name == '20newsgroups':
                        path = f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/20newsgroups_ood/optimal_hyperparameters.yaml'
                    else:
                        path = f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/sst2/optimal_hyperparameters.yaml'
                    with open(path, 'r') as f:
                        params = yaml.load(f, Loader=yaml.FullLoader)

                    num_train_epochs = params['num_train_epochs']
                    args += f' training.num_train_epochs\={num_train_epochs}'
                    learning_rate = params['learning_rate']
                    args += f' training.learning_rate\={learning_rate}'
                    per_device_train_batch_size = params['per_device_train_batch_size']
                    args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                    weight_decay = params['weight_decay']
                    args += f' +training.weight_decay\={weight_decay}'

                    new_task += f" args='{args}'"
                    new_task += f' task_configs={task_configs}'

                    new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}'

                    full_config += '\n' + new_task if len(full_config) else new_task
                except:
                    pass
    with open (f'../../scripts/ood_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [39]:
tasks = ['sst2', '20newsgroups']

train_configs = {
    'script': ['run_ood.py'],
    'noise_perc': [0.01, 0.05, 0.1, 0.15, 0.2, 0.3],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_new_benchmark_noise_sngp.sh')

In [41]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))
    
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('data.ood_data\={}'.format(config['ood_data']))
    config_list.append('+data.noise_perc\={}'.format(config['noise_perc']))
    
    config_list.append('data.validation_subsample\=0.0')
    config_list.append('training\=electra_base')
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                try:
                    task_configs = f'{task_name}_ood.yaml'

                    new_task = copy.deepcopy(base_arg)
                    new_task += f' cuda_devices={cuda_devices}'
                    script = conf['script']
                    new_task += f' script={script}'

                    noise_perc = conf['noise_perc']

                    args = ' '.join(generate_func[i](conf))

                    reg = 'raw' 
                    use_spectralnorm = 'sngp'
                    
                    if task_name == '20newsgroups':
                        path = f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/20newsgroups_ood/optimal_hyperparameters.yaml'
                    else:
                        path = f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/sst2/optimal_hyperparameters.yaml'
                    with open(path, 'r') as f:
                        params = yaml.load(f, Loader=yaml.FullLoader)

                    num_train_epochs = params['num_train_epochs']
                    args += f' training.num_train_epochs\={num_train_epochs}'
                    learning_rate = params['learning_rate']
                    args += f' training.learning_rate\={learning_rate}'
                    per_device_train_batch_size = params['per_device_train_batch_size']
                    args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                    weight_decay = params['weight_decay']
                    args += f' +training.weight_decay\={weight_decay}'

                    args += f' training.per_device_eval_batch_size\=256'

                    new_task += f" args='{args}'"
                    new_task += f' config_path=../configs/{task_configs}'

                    ood_data = conf['ood_data']
                    if task_name == ood_data:
                        continue

                    base_path=f' output_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}/{ood_data}'

                    ue_type = conf['ue']
                    base_path+= f'/{ue_type}'
                    new_task+=base_path

                    if task_name == '20newsgroups':
                        model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}/models/{task_name}_ood'
                    else:
                        model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}/models/{task_name}'
                    new_task+=f' model_series_dir={model_series_dir}'

                    calc_output_dir = f'../workdir/run_calc_ues_metrics/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}/{ood_data}/{ue_type}'
                    calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics.py runs_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}/{ood_data}/{ue_type} extract_config=False output_dir={calc_output_dir};'

                    full_config += '\n' + new_task if len(full_config) else new_task
                    full_config += '\n' + calc_command
                except:
                    pass
    with open (f'../../scripts/ood_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [42]:
tasks = ['sst2', '20newsgroups']

ue_configs = {
    'ue': ['sngp'],
    'script': ['run_ood.py'],
    'ood_data': ['imdb', 'trec', 'wmt16', 'amazon', 'mnli', 'rte', '20newsgroups', 'sst2'],
    'noise_perc': [0.01, 0.05, 0.1, 0.15, 0.2, 0.3],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_new_benchmark_noise_sngp.sh')

In [43]:
full_test = 'bash run_train_electra_models_new_benchmark_noise_sngp.sh\n'
full_test += 'bash run_eval_electra_new_benchmark_noise_sngp.sh'

with open (f'../../scripts/ood_scripts/run_exps_new_benchmark_noise_sngp.sh', 'w') as rsh:
    rsh.write(full_test)

In [44]:
full_test = 'bash run_exps_new_benchmark_noise.sh\n'
full_test += 'bash run_exps_new_benchmark_noise_sngp.sh'

with open (f'../../scripts/ood_scripts/run_all_exps_new_benchmark_noise.sh', 'w') as rsh:
    rsh.write(full_test)

# OOD exps with old benchmark methods

In [49]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc')
    config_list.append('do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.calibrate\={}'.format(config['calibrate']))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    
    if config['use_spectralnorm'] == 'True':
        config_list.append('+ue.n_power_iterations\={}'.format(config['n_power_iterations']))

    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    
    config_list.append('training\=electra_base')
    return config_list

In [50]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                try:
                    task_configs = f'{task_name}.yaml'

                    new_task = copy.deepcopy(base_arg)
                    new_task += f' cuda_devices={cuda_devices} seeds=[23419,705525,4837,10671619,1084218,43]'
                    script = conf['script']
                    new_task += f' script={script}'
                    args = ' '.join(generate_func[i](conf))

                    reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                    use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'

                    with open(f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                        params = yaml.load(f, Loader=yaml.FullLoader)

                    num_train_epochs = params['num_train_epochs']
                    args += f' training.num_train_epochs\={num_train_epochs}'
                    learning_rate = params['learning_rate']
                    args += f' training.learning_rate\={learning_rate}'
                    per_device_train_batch_size = params['per_device_train_batch_size']
                    args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                    weight_decay = params['weight_decay']
                    args += f' +training.weight_decay\={weight_decay}'

                    margin = params['margin']
                    lamb_intra = params['lamb_intra']
                    lamb = params['lamb']
                    args += f' +ue.margin\={margin}'
                    args += f' +ue.lamb_intra\={lamb_intra}'
                    args += f' ue.lamb\={lamb}'

                    new_task += f" args='{args}'"
                    new_task += f' task_configs={task_configs}'
                    subsample_perc = conf['subsample_perc']

                    new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/subsample_perc_{subsample_perc}'

                    full_config += '\n' + new_task if len(full_config) else new_task
                except:
                    pass
    with open (f'../../scripts/ood_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [51]:
tasks = ['clinc']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['raw'],
    'use_spectralnorm': ['False', 'True'],
    'script': ['run_ood.py'],
    'subsample_perc': [0.0, 0.01, 0.02, 0.05, 0.1, 0.15, 0.2],
    'n_power_iterations': [10],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_subsamples_clinc.sh')

In [52]:
tasks = ['rostd']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['raw'],
    'use_spectralnorm': ['False', 'True'],
    'script': ['run_ood.py'],
    'subsample_perc': [0.0, 0.005, 0.01, 0.015, 0.02, 0.03, 0.04],
    'n_power_iterations': [10],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_subsamples_rostd.sh')

### sngp

In [53]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=sngp')
    config_list.append('do_ue_estimate\=False')

    config_list.append('ue.calibrate\={}'.format(config['calibrate']))
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    
    config_list.append('training\=electra_base')
    return config_list

In [54]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                try:
                    task_configs = f'{task_name}.yaml'

                    new_task = copy.deepcopy(base_arg)
                    new_task += f' cuda_devices={cuda_devices}'
                    script = conf['script']
                    new_task += f' script={script} seeds=[23419,705525,4837,10671619,1084218,43]'
                    args = ' '.join(generate_func[i](conf))

                    reg = 'raw'
                    use_spectralnorm = 'sngp'

                    with open(f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                        params = yaml.load(f, Loader=yaml.FullLoader)

                    num_train_epochs = params['num_train_epochs']
                    args += f' training.num_train_epochs\={num_train_epochs}'
                    learning_rate = params['learning_rate']
                    args += f' training.learning_rate\={learning_rate}'
                    per_device_train_batch_size = params['per_device_train_batch_size']
                    args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                    weight_decay = params['weight_decay']
                    args += f' +training.weight_decay\={weight_decay}'

                    new_task += f" args='{args}'"
                    new_task += f' task_configs={task_configs}'
                    subsample_perc = conf['subsample_perc']

                    new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/subsample_perc_{subsample_perc}'

                    full_config += '\n' + new_task if len(full_config) else new_task
                except:
                    pass
    with open (f'../../scripts/ood_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [55]:
tasks = ['clinc']

train_configs = {
    'calibrate': ['False'],
    'script': ['run_ood.py'],
    'subsample_perc': [0.0, 0.01, 0.02, 0.05, 0.1, 0.15, 0.2],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_subsamples_clinc_sngp.sh')

tasks = ['rostd']

train_configs = {
    'calibrate': ['False'],
    'script': ['run_ood.py'],
    'subsample_perc': [0.0, 0.005, 0.01, 0.015, 0.02, 0.03, 0.04],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_subsamples_rostd_sngp.sh')

### eval

In [74]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))
    config_list.append('++ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))

    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))

    config_list.append('do_ue_estimate\=True')
    if config['ue']=='mc':
        config_list.append('ue.dropout_subs\=all')
    
    if config['ue'] == 'mc_mahalanobis':
        config_list.append('ue.use_cache\=False')
    else:
        config_list.append('ue.use_cache\=True')
    
    config_list.append('ue.calibrate\=False')

    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    
    if config['ue'] == 'nuq' or config['ue'] == 'l_nuq':
        config_list.append('ue.nuq.n_neighbors\=60')
        config_list.append('ue.nuq.log_pN\=0')
        config_list.append('+ue.nuq.n_points\=50')
        config_list.append('+ue.nuq.n_folds\=20')
        config_list.append('+ue.nuq.n_samples\=5')
    
    config_list.append('training\=electra_base')
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                if conf['ue'] == 'mc' and conf['use_spectralnorm'] == 'True':
                    continue

                try:
                    task_configs = f'{task_name}.yaml'

                    new_task = copy.deepcopy(base_arg)
                    new_task += f' cuda_devices={cuda_devices}'
                    script = conf['script']
                    new_task += f' script={script}'

                    args = ' '.join(generate_func[i](conf))

                    reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                    use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'

                    with open(f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                        params = yaml.load(f, Loader=yaml.FullLoader)

                    num_train_epochs = params['num_train_epochs']
                    args += f' training.num_train_epochs\={num_train_epochs}'
                    learning_rate = params['learning_rate']
                    args += f' training.learning_rate\={learning_rate}'
                    per_device_train_batch_size = params['per_device_train_batch_size']
                    args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                    weight_decay = params['weight_decay']
                    args += f' +training.weight_decay\={weight_decay}'

                    margin = params['margin']
                    lamb_intra = params['lamb_intra']
                    lamb = params['lamb']
                    args += f' ++ue.margin\={margin}'
                    args += f' ++ue.lamb_intra\={lamb_intra}'
                    args += f' ue.lamb\={lamb}'

                    args += f' training.per_device_eval_batch_size\=64'

                    new_task += f" args='{args}'"
                    new_task += f' config_path=../configs/{task_configs}'

                    reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                    use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                    subsample_perc = conf['subsample_perc']

                    base_path=f' output_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/subsample_perc_{subsample_perc}'

                    ue_type = conf['ue']
                    base_path+= f'/{ue_type}'
                    new_task+=base_path

                    model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/subsample_perc_{subsample_perc}/models/{task_name}'
                    new_task+=f' model_series_dir={model_series_dir}'

                    full_config += '\n' + new_task if len(full_config) else new_task
                except:
                    pass
    with open (f'../../scripts/ood_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [59]:
tasks = ['clinc']

ue_configs = {
    'ue': ['l_nuq', 'nuq', 'mahalanobis', 'l_mahalanobis', 'mc'],
    'use_spectralnorm': ['False', 'True'],
    'reg_type': ['raw'],
    'script': ['run_ood.py'],
    'subsample_perc': [0.0, 0.01, 0.02, 0.05, 0.1, 0.15, 0.2],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_subsamples_clinc.sh')

In [60]:
tasks = ['rostd']

ue_configs = {
    'ue': ['l_nuq', 'nuq', 'mahalanobis', 'l_mahalanobis', 'mc'],
    'use_spectralnorm': ['False', 'True'],
    'reg_type': ['raw'],
    'script': ['run_ood.py'],
    'subsample_perc': [0.0, 0.005, 0.01, 0.015, 0.02, 0.03, 0.04],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_subsamples_rostd.sh')

In [61]:
full_test = 'bash run_train_electra_models_subsamples_clinc.sh\n'
full_test += 'bash run_train_electra_models_subsamples_rostd.sh\n'
full_test += 'bash run_eval_electra_subsamples_clinc.sh\n'
full_test += 'bash run_eval_electra_subsamples_rostd.sh'

with open (f'../../scripts/ood_scripts/run_exps_subsamples.sh', 'w') as rsh:
    rsh.write(full_test)

### eval sngp

In [63]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))

    config_list.append('do_ue_estimate\=True')
    config_list.append('ue.calibrate\=False')

    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    config_list.append('training\=electra_base')
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                try:
                    task_configs = f'{task_name}.yaml'

                    new_task = copy.deepcopy(base_arg)
                    new_task += f' cuda_devices={cuda_devices}'
                    script = conf['script']
                    new_task += f' script={script} seeds=[23419,705525,4837,10671619,1084218,43]'

                    args = ' '.join(generate_func[i](conf))

                    reg = 'raw'
                    use_spectralnorm = 'sngp'

                    with open(f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                        params = yaml.load(f, Loader=yaml.FullLoader)

                    num_train_epochs = params['num_train_epochs']
                    args += f' training.num_train_epochs\={num_train_epochs}'
                    learning_rate = params['learning_rate']
                    args += f' training.learning_rate\={learning_rate}'
                    per_device_train_batch_size = params['per_device_train_batch_size']
                    args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                    weight_decay = params['weight_decay']
                    args += f' +training.weight_decay\={weight_decay}'

                    args += f' training.per_device_eval_batch_size\=64'

                    new_task += f" args='{args}'"
                    new_task += f' config_path=../configs/{task_configs}'

                    subsample_perc = conf['subsample_perc']

                    base_path=f' output_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/subsample_perc_{subsample_perc}'

                    ue_type = conf['ue']
                    base_path+= f'/{ue_type}'
                    new_task+=base_path

                    model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/subsample_perc_{subsample_perc}/models/{task_name}'
                    new_task+=f' model_series_dir={model_series_dir}'

                    full_config += '\n' + new_task if len(full_config) else new_task
                except:
                    pass
    with open (f'../../scripts/ood_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [65]:
tasks = ['clinc']

ue_configs = {
    'ue': ['sngp'],
    'script': ['run_ood.py'],
    'subsample_perc': [0.0, 0.01, 0.02, 0.05, 0.1, 0.15, 0.2],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_subsamples_clinc_sngp.sh')


tasks = ['rostd']

ue_configs = {
    'ue': ['sngp'],
    'script': ['run_ood.py'],
    'subsample_perc': [0.0, 0.005, 0.01, 0.015, 0.02, 0.03, 0.04],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_subsamples_rostd_sngp.sh')

In [66]:
full_test = 'bash run_train_electra_models_subsamples_clinc_sngp.sh\n'
full_test += 'bash run_train_electra_models_subsamples_rostd_sngp.sh\n'
full_test += 'bash run_eval_electra_subsamples_clinc_sngp.sh\n'
full_test += 'bash run_eval_electra_subsamples_rostd_sngp.sh'

with open (f'../../scripts/ood_scripts/run_exps_subsamples_sngp.sh', 'w') as rsh:
    rsh.write(full_test)

In [146]:
full_test = 'bash run_exps_subsamples.sh\n'
full_test += 'bash run_exps_subsamples_sngp.sh'

with open (f'../../scripts/ood_scripts/run_all_exps_subsamples.sh', 'w') as rsh:
    rsh.write(full_test)

## Noise

In [143]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc')
    config_list.append('do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.calibrate\={}'.format(config['calibrate']))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    if config['use_spectralnorm'] == 'True':
        config_list.append('+ue.n_power_iterations\={}'.format(config['n_power_iterations']))

    config_list.append('+data.noise_perc\={}'.format(config['noise_perc']))
    
    config_list.append('training\=electra_base')
    return config_list

In [144]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script} seeds=[23419,705525,4837,10671619,1084218,43]'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                with open(f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' +ue.margin\={margin}'
                args += f' +ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'

                noise_perc = conf['noise_perc']
                new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../../scripts/ood_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [145]:
tasks = ['clinc', 'rostd']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['raw'],
    'use_spectralnorm': ['False', 'True'],
    'script': ['run_ood.py'],
    'noise_perc': [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3],
    'n_power_iterations': [10],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_ood_models_noise.sh')

### sngp

In [70]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=sngp')
    config_list.append('do_ue_estimate\=False')
    
    config_list.append('ue.calibrate\={}'.format(config['calibrate']))
    config_list.append('+data.noise_perc\={}'.format(config['noise_perc']))
    
    config_list.append('training\=electra_base')
    return config_list

In [71]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script} seeds=[23419,705525,4837,10671619,1084218,43]'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'raw'
                use_spectralnorm = 'sngp' 
                
                with open(f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'

                noise_perc = conf['noise_perc']
                new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../../scripts/ood_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [72]:
tasks = ['clinc', 'rostd']

train_configs = {
    'calibrate': ['False'],
    'script': ['run_ood.py'],
    'noise_perc': [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_ood_models_noise_sngp.sh')

### eval

In [75]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))
    config_list.append('++ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))

    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))

    config_list.append('do_ue_estimate\=True')
    
    if config['ue']=='mc':
        config_list.append('ue.dropout_subs\=all')
    
    if config['ue'] == 'mc_mahalanobis':
        config_list.append('ue.use_cache\=False')
    else:
        config_list.append('ue.use_cache\=True')
    
    config_list.append('ue.calibrate\=False')
    config_list.append('+data.noise_perc\={}'.format(config['noise_perc']))
    
    config_list.append('training\=electra_base')
    
    if config['ue'] == 'nuq' or config['ue'] == 'l_nuq':
        config_list.append('ue.nuq.n_neighbors\=60')
        config_list.append('+ue.nuq.n_points\=50')
        config_list.append('+ue.nuq.n_folds\=20')
        config_list.append('+ue.nuq.n_samples\=5')
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                if conf['ue'] == 'mc' and conf['use_spectralnorm'] == 'True':
                    continue
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script}'
                
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                with open(f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' ++ue.margin\={margin}'
                args += f' ++ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'

                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                noise_perc = conf['noise_perc']
                base_path=f' output_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}'
                
                ue_type = conf['ue'] 
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                calc_output_dir = f'../workdir/run_calc_ues_metrics/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}/{ue_type}'
                calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics.py runs_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}/{ue_type} extract_config=False output_dir={calc_output_dir};'

                full_config += '\n' + new_task if len(full_config) else new_task
                full_config += '\n' + calc_command
                        
    with open (f'../../scripts/ood_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [76]:
tasks = ['clinc', 'rostd']

ue_configs = {
    'ue': ['l_mahalanobis', 'mahalanobis', 'nuq', 'l_nuq', 'mc'],
    'use_spectralnorm': ['False', 'True'],
    'reg_type': ['raw'],
    'script': ['run_ood.py'],
    'noise_perc': [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_ood_models_noise.sh')

In [77]:
full_test = 'bash run_train_electra_ood_models_noise.sh\n'
full_test += 'bash run_eval_electra_ood_models_noise.sh'

with open (f'../../scripts/ood_scripts/run_exps_ood_noise.sh', 'w') as rsh:
    rsh.write(full_test)

### eval sngp

In [78]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))
    config_list.append('do_ue_estimate\=True')

    config_list.append('ue.calibrate\=False')
    config_list.append('+data.noise_perc\={}'.format(config['noise_perc']))
    
    config_list.append('training\=electra_base')
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script}'
                
                args = ' '.join(generate_func[i](conf))
                
                reg = 'raw'
                use_spectralnorm = 'sngp'
                
                with open(f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
        
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'

                noise_perc = conf['noise_perc']
                base_path=f' output_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}'
                
                ue_type = conf['ue'] 
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                calc_output_dir = f'../workdir/run_calc_ues_metrics/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}/{ue_type}'
                calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics.py runs_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}/{ue_type} extract_config=False output_dir={calc_output_dir};'

                full_config += '\n' + new_task if len(full_config) else new_task
                full_config += '\n' + calc_command
                        
    with open (f'../../scripts/ood_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [79]:
tasks = ['clinc', 'rostd']

ue_configs = {
    'ue': ['sngp'],
    'use_spectralnorm': ['False'],
    'reg_type': ['raw'],
    'script': ['run_ood.py'],
    'noise_perc': [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_ood_models_noise_sngp.sh')

In [80]:
full_test = 'bash run_train_electra_ood_models_noise_sngp.sh\n'
full_test += 'bash run_eval_electra_ood_models_noise_sngp.sh'

with open (f'../../scripts/ood_scripts/run_exps_ood_noise_sngp.sh', 'w') as rsh:
    rsh.write(full_test)

In [81]:
full_test = 'bash run_exps_ood_noise_sngp.sh\n'
full_test += 'bash run_exps_subsamples_sngp.sh\n'
full_test += 'bash run_exps_new_benchmark_subsamples_sngp.sh'

with open (f'../../scripts/ood_scripts/run_sngp_subsample_noise_exps.sh', 'w') as rsh:
    rsh.write(full_test)

In [147]:
full_test = 'bash run_exps_ood_noise.sh\n'
full_test += 'bash run_exps_ood_noise_sngp.sh'

with open (f'../../scripts/ood_scripts/run_all_exps_ood_noise.sh', 'w') as rsh:
    rsh.write(full_test)

# Train MISCL datasets with different size

In [2]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc')
    config_list.append('do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.calibrate\={}'.format(config['calibrate']))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    if config['use_spectralnorm'] == 'True':
        config_list.append('+ue.n_power_iterations\={}'.format(config['n_power_iterations']))
    
    config_list.append('data.validation_subsample\=0.0')
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    config_list.append('training\=electra_base')
    return config_list

In [3]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script} seeds=[23419,705525,4837,10671619,1084218,43]'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                use_spectralnorm_params = '_sn' if conf['use_spectralnorm'] == 'True' else ''

                path = f'../../optimal_hyperparameters/electra/raw{use_spectralnorm_params}_{task_name}_optimal_hyperparameters.yaml'
                with open(path, 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' +ue.margin\={margin}'
                args += f' +ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                subsample_perc = conf['subsample_perc']
                new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/subsample_perc_{subsample_perc}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [4]:
tasks = ['sst2']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['raw'],
    'use_spectralnorm': ['False', 'True'],
    'script': ['run_glue.py'],
    'subsample_perc': [0.001, 0.005, 0.01, 0.015, 0.02, 0.025, 0.03],
    'n_power_iterations': [10],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_sst2_dif_size.sh')

In [5]:
tasks = ['mrpc']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['raw'],
    'use_spectralnorm': ['False', 'True'],
    'script': ['run_glue.py'],
    'subsample_perc': [0.01, 0.02, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4],
    'n_power_iterations': [10],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_mrpc_dif_size.sh')

In [6]:
tasks = ['cola']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['raw'],
    'use_spectralnorm': ['False', 'True'],
    'script': ['run_glue.py'],
    'validation_subsample': [0.0],
    'subsample_perc': [0.01, 0.02, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4],
    'n_power_iterations': [10],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_cola_dif_size.sh')

In [7]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))
    config_list.append('++ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))

    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))

    config_list.append('do_ue_estimate\=True')
    if config['ue']=='mc':
        config_list.append('ue.dropout_subs\=all')
    
    if config['ue'] == 'mc_mahalanobis':
        config_list.append('ue.use_cache\=False')
    else:
        config_list.append('ue.use_cache\=True')
    
    config_list.append('ue.calibrate\=False')
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    if config['ue'] == 'nuq' or config['ue'] == 'l_nuq':
        config_list.append('ue.nuq.n_neighbors\=60')
        config_list.append('ue.nuq.log_pN\=-40')
        config_list.append('+ue.nuq.n_points\=50')
        config_list.append('+ue.nuq.n_folds\=20')
        config_list.append('+ue.nuq.n_samples\=5')
    
    config_list.append('data.validation_subsample\=0.0')
    config_list.append('training\=electra_base')
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                if conf['ue'] == 'mc' and conf['use_spectralnorm'] == 'True':
                    continue
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script}'
                
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                use_spectralnorm_params = '_sn' if conf['use_spectralnorm'] == 'True' else ''
                
                path = f'../../optimal_hyperparameters/electra/raw{use_spectralnorm_params}_{task_name}_optimal_hyperparameters.yaml'
                with open(path, 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)


                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' ++ue.margin\={margin}'
                args += f' ++ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'

                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                subsample_perc = conf['subsample_perc']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                subsample_perc = conf['subsample_perc']
                base_path=f' output_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/subsample_perc_{subsample_perc}'
                
                ue_type = conf['ue']
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/subsample_perc_{subsample_perc}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                calc_output_dir = f'../workdir/run_calc_ues_metrics/electra_{reg}_{use_spectralnorm}/{task_name}/subsample_perc_{subsample_perc}/{ue_type}'
                calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics.py runs_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/subsample_perc_{subsample_perc}/{ue_type} extract_config=False output_dir={calc_output_dir};'

                full_config += '\n' + new_task if len(full_config) else new_task
                full_config += '\n' + calc_command
            
    with open (f'../../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [8]:
tasks = ['sst2']

ue_configs = {
    'ue': ['l_mahalanobis', 'mahalanobis', 'nuq', 'l_nuq', 'mc'],
    'use_spectralnorm': ['False', 'True'],
    'reg_type': ['raw'],
    'script': ['run_glue.py'],
    'subsample_perc': [0.001, 0.005, 0.01, 0.015, 0.02, 0.025, 0.03],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_models_sst2_det_best_dif_size.sh')

In [9]:
tasks = ['mrpc']

ue_configs = {
    'ue': ['l_mahalanobis', 'mahalanobis', 'nuq', 'l_nuq', 'mc'],
    'use_spectralnorm': ['False', 'True'],
    'reg_type': ['raw'],
    'script': ['run_glue.py'],
    'subsample_perc': [0.01, 0.02, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_models_mrpc_det_best_dif_size.sh')

In [10]:
tasks = ['cola']

ue_configs = {
    'ue': ['l_mahalanobis', 'mahalanobis', 'nuq', 'l_nuq', 'mc'],
    'use_spectralnorm': ['False', 'True'],
    'reg_type': ['raw'],
    'script': ['run_glue.py'],
    'subsample_perc': [0.01, 0.02, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_models_cola_det_best_dif_size.sh')

In [11]:
full_test = 'bash run_train_electra_models_sst2_dif_size.sh\n'
full_test += 'bash run_train_electra_models_mrpc_dif_size.sh\n'
full_test += 'bash run_train_electra_models_cola_dif_size.sh\n'

full_test += 'bash run_eval_electra_models_sst2_det_best_dif_size.sh\n'
full_test += 'bash run_eval_electra_models_mrpc_det_best_dif_size.sh\n'
full_test += 'bash run_eval_electra_models_cola_det_best_dif_size.sh'

with open (f'../../scripts/miscl_scripts/run_exps_diff_size.sh', 'w') as rsh:
    rsh.write(full_test)

### sngp

In [247]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=sngp')
    config_list.append('do_ue_estimate\=False')

    config_list.append('ue.calibrate\={}'.format(config['calibrate']))

    config_list.append('data.validation_subsample\=0.0')
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    config_list.append('training\=electra_base')
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script} seeds=[23419,705525,4837,10671619,1084218,43]'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'raw'
                use_spectralnorm = 'sngp' 
                
                with open(f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                subsample_perc = conf['subsample_perc']
                new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/subsample_perc_{subsample_perc}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [248]:
tasks = ['sst2']

train_configs = {
    'calibrate': ['False'],
    'script': ['run_glue.py'],
    'subsample_perc': [0.001, 0.005, 0.01, 0.015, 0.02, 0.025, 0.03],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_sst2_dif_size_sngp.sh')

In [249]:
tasks = ['cola']

train_configs = {
    'calibrate': ['False'],
    'script': ['run_glue.py'],
    'subsample_perc': [0.01, 0.02, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_cola_dif_size_sngp.sh')

In [250]:
tasks = ['mrpc']

train_configs = {
    'calibrate': ['False'],
    'script': ['run_glue.py'],
    'subsample_perc': [0.01, 0.02, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_mrpc_dif_size_sngp.sh')

In [251]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('ue.calibrate\=False')
    config_list.append('data.subsample_perc\={}'.format(config['subsample_perc']))
    
    config_list.append('data.validation_subsample\=0.0')
    config_list.append('training\=electra_base')
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script}'
                
                args = ' '.join(generate_func[i](conf))
                
                reg = 'raw'
                use_spectralnorm = 'sngp' 
                
                with open(f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
        
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'

                subsample_perc = conf['subsample_perc']
                base_path=f' output_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/subsample_perc_{subsample_perc}'
                
                ue_type = conf['ue']
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/subsample_perc_{subsample_perc}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                calc_output_dir = f'../workdir/run_calc_ues_metrics/electra_{reg}_{use_spectralnorm}/{task_name}/subsample_perc_{subsample_perc}/{ue_type}'
                calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics.py runs_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/subsample_perc_{subsample_perc}/{ue_type} extract_config=False output_dir={calc_output_dir};'

                full_config += '\n' + new_task if len(full_config) else new_task
                full_config += '\n' + calc_command
            
    with open (f'../../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [252]:
tasks = ['sst2']

ue_configs = {
    'ue': ['sngp'],
    'script': ['run_glue.py'],
    'subsample_perc': [0.001, 0.005, 0.01, 0.015, 0.02, 0.025, 0.03],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_models_sst2_det_best_dif_size_sngp.sh')

In [253]:
tasks = ['cola']

ue_configs = {
    'ue': ['sngp'],
    'script': ['run_glue.py'],
    'subsample_perc': [0.01, 0.02, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_models_cola_det_best_dif_size_sngp.sh')

In [254]:
tasks = ['mrpc']

ue_configs = {
    'ue': ['sngp'],
    'script': ['run_glue.py'],
    'subsample_perc': [0.01, 0.02, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_models_mrpc_det_best_dif_size_sngp.sh')

In [255]:
full_test = 'bash run_train_electra_models_sst2_dif_size_sngp.sh\n'
full_test += 'bash run_train_electra_models_mrpc_dif_size_sngp.sh\n'
full_test += 'bash run_train_electra_models_cola_dif_size_sngp.sh\n'

full_test += 'bash run_eval_electra_models_sst2_det_best_dif_size_sngp.sh\n'
full_test += 'bash run_eval_electra_models_mrpc_det_best_dif_size_sngp.sh\n'
full_test += 'bash run_eval_electra_models_cola_det_best_dif_size_sngp.sh'

with open (f'../../scripts/miscl_scripts/run_exps_diff_size_sngp.sh', 'w') as rsh:
    rsh.write(full_test)

# With Noise

In [12]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc')
    config_list.append('do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('ue.calibrate\={}'.format(config['calibrate']))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    if config['use_spectralnorm'] == 'True':
        config_list.append('+ue.n_power_iterations\={}'.format(config['n_power_iterations']))

    config_list.append('data.validation_subsample\=0.0')
    config_list.append('+data.noise_perc\={}'.format(config['noise_perc']))
    
    config_list.append('training\=electra_base')
    return config_list

In [13]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script} seeds=[23419,705525,4837,10671619,1084218,43]'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                use_spectralnorm_params = '_sn' if conf['use_spectralnorm'] == 'True' else ''
                
                path = f'../../optimal_hyperparameters/electra/raw{use_spectralnorm_params}_{task_name}_optimal_hyperparameters.yaml'
                with open(path, 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' +ue.margin\={margin}'
                args += f' +ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'

                noise_perc = conf['noise_perc']
                new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [14]:
tasks = ['mrpc', 'cola', 'sst2']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['raw'],
    'use_spectralnorm': ['False', 'True'],
    'script': ['run_glue.py'],
    'noise_perc': [0.0, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3],
    'n_power_iterations': [10],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_noise.sh')

In [15]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))
    config_list.append('++ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))

    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    if config['ue']=='mc':
        config_list.append('ue.dropout_subs\=all')

    config_list.append('do_ue_estimate\=True')
    
    if config['ue'] == 'mc_mahalanobis' or config['ue'] == 'mc':
        config_list.append('ue.use_cache\=False')
    else:
        config_list.append('ue.use_cache\=True')
    
    config_list.append('data.validation_subsample\=0.0')
    config_list.append('ue.calibrate\=False')
    config_list.append('+data.noise_perc\={}'.format(config['noise_perc']))
    
    config_list.append('training\=electra_base')
    
    if config['ue'] == 'nuq' or config['ue'] == 'l_nuq':
        config_list.append('ue.nuq.n_neighbors\=60')
        config_list.append('+ue.nuq.n_points\=50')
        config_list.append('+ue.nuq.n_folds\=20')
        config_list.append('+ue.nuq.n_samples\=5')
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                if conf['ue'] == 'mc' and conf['use_spectralnorm'] == 'True':
                    continue
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script}'
                
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                use_spectralnorm_params = '_sn' if conf['use_spectralnorm'] == 'True' else ''
                
                path = f'../../optimal_hyperparameters/electra/raw{use_spectralnorm_params}_{task_name}_optimal_hyperparameters.yaml'
                with open(path, 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' ++ue.margin\={margin}'
                args += f' ++ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'

                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                noise_perc = conf['noise_perc']
                base_path=f' output_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}'
                
                ue_type = conf['ue'] 
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                calc_output_dir = f'../workdir/run_calc_ues_metrics/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}/{ue_type}'
                calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics.py runs_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}/{ue_type} extract_config=False output_dir={calc_output_dir};'

                full_config += '\n' + new_task if len(full_config) else new_task
                full_config += '\n' + calc_command
                        
    with open (f'../../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [16]:
tasks = ['mrpc', 'cola', 'sst2']

ue_configs = {
    'ue': ['l_mahalanobis', 'mahalanobis', 'nuq', 'l_nuq', 'mc'],
    'use_spectralnorm': ['False', 'True'],
    'reg_type': ['raw'],
    'script': ['run_glue.py'],
    'noise_perc': [0.0, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_models_noise.sh')

In [17]:
full_test = 'bash run_train_electra_models_noise.sh\n'
full_test += 'bash run_eval_electra_models_noise.sh'

with open (f'../../scripts/miscl_scripts/run_exps_noise.sh', 'w') as rsh:
    rsh.write(full_test)

### sngp

In [229]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=sngp')
    config_list.append('do_ue_estimate\=False')

    config_list.append('ue.calibrate\={}'.format(config['calibrate']))
    config_list.append('+data.noise_perc\={}'.format(config['noise_perc']))
    
    config_list.append('data.validation_subsample\=0.0')
    config_list.append('training\=electra_base')
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script} seeds=[23419,705525,4837,10671619,1084218,43]'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'raw'
                use_spectralnorm = 'sngp' 
                
                with open(f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'

                noise_perc = conf['noise_perc']
                new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [230]:
tasks = ['mrpc', 'cola', 'sst2']

train_configs = {
    'calibrate': ['False'],
    'script': ['run_glue.py'],
    'noise_perc': [0.0, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_electra_models_noise_sngp.sh')

In [231]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('ue.calibrate\=False')
    config_list.append('+data.noise_perc\={}'.format(config['noise_perc']))
    
    config_list.append('data.validation_subsample\=0.0')
    config_list.append('training\=electra_base')
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script}'
                
                args = ' '.join(generate_func[i](conf))
                
                reg = 'raw'
                use_spectralnorm = 'sngp' 
                
                with open(f'../../workdir/hp_search/electra_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
    
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'

                noise_perc = conf['noise_perc']
                base_path=f' output_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}'
                
                ue_type = conf['ue'] 
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                calc_output_dir = f'../workdir/run_calc_ues_metrics/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}/{ue_type}'
                calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics.py runs_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/noise_perc_{noise_perc}/{ue_type} extract_config=False output_dir={calc_output_dir};'

                full_config += '\n' + new_task if len(full_config) else new_task
                full_config += '\n' + calc_command
                        
    with open (f'../../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [232]:
tasks = ['mrpc', 'cola', 'sst2']

ue_configs = {
    'ue': ['sngp'],
    'script': ['run_glue.py'],
    'noise_perc': [0.0, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_models_noise_sngp.sh')

In [233]:
full_test = 'bash run_train_electra_models_noise_sngp.sh\n'
full_test += 'bash run_eval_electra_models_noise_sngp.sh'

with open (f'../../scripts/miscl_scripts/run_exps_noise_sngp.sh', 'w') as rsh:
    rsh.write(full_test)

In [234]:
full_test = 'bash run_exps_noise_sngp.sh\n'
full_test += 'bash run_exps_diff_size_sngp.sh'

with open (f'../../scripts/miscl_scripts/run_exps_noise_size_sngp.sh', 'w') as rsh:
    rsh.write(full_test)

In [235]:
full_test = 'bash run_exps_diff_size.sh\n'
full_test += 'bash run_exps_diff_size_sngp.sh'

with open (f'../../scripts/miscl_scripts/run_all_exps_diff_size.sh', 'w') as rsh:
    rsh.write(full_test)

In [236]:
full_test = 'bash run_exps_noise.sh\n'
full_test += 'bash run_exps_noise_sngp.sh'

with open (f'../../scripts/miscl_scripts/run_all_exps_noise.sh', 'w') as rsh:
    rsh.write(full_test)